# 7.3 Invariance Over Time
## Measurement invariance across time for quantitative item responses

In [ ]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Rpy2 imports
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri, numpy2ri
from rpy2.robjects.packages import importr

# Automatic conversion of arrays and dataframes
pandas2ri.activate()
numpy2ri.activate()

# Set random seed for reproducibility
ro.r('set.seed(123)')

# Ipython extenrsion for magix plotting
%load_ext rpy2.ipython

# R imports
importr('base')
importr('lavaan')
importr('semTools')
importr('MPsychoR')
importr('lordif')
importr('Hmisc')

rpy2.robjects.packages.Package as a <module 'Hmisc'>

CFA can also be extended to longitudinal settings, where indicators are presented
to the same individual at multiple points in time $t$. Across the time points,
we can consider the same measurement invariance principles as in multigroup CFA
(configural, weak, and strong invariance). However, in longitudinal CFA we need
to account for **correlated residuals** (i.e. autoregressive effects) since we cannot assume that independence holds
across time points.

### Load, prepare and inspect the dataset

In [19]:
ro.r('data("SDOwave")')
# Convert to Python
SDOwave = pandas2ri.rpy2py(ro.globalenv['SDOwave'])
print(SDOwave.head())

   I1.1996  I2.1996  I3.1996  I4.1996  I1.1997  I2.1997  I3.1997  I4.1997  \
1      2.0      2.0      4.0      4.0      1.0      1.0      4.0      1.0   
2      1.0      1.0      2.0      2.0      1.0      1.0      2.0      3.0   
3      1.0      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
4      2.0      2.0      3.0      3.0      2.0      2.0      2.0      2.0   
5      1.0      1.0      6.0      4.0      7.0      1.0      1.0      1.0   

   I1.1998  I2.1998  I3.1998  I4.1998  I1.1999  I2.1999  I3.1999  I4.1999  \
1      2.0      1.0      4.0      2.0      2.0      3.0      3.0      1.0   
2      4.0      2.0      5.0      4.0      3.0      2.0      4.0      4.0   
3      1.0      1.0      2.0      1.0      1.0      1.0      6.0      1.0   
4      2.0      2.0      2.0      2.0      2.0      1.0      1.0      1.0   
5      1.0      1.0      4.0      1.0      4.0      1.0      3.0      2.0   

   I1.2000  I2.2000  I3.2000  I4.2000  
1      2.0      1.0      5.0      

#### The dataset

To illustrate longitudinal CFA, we use a dataset on social dominance orientation
(SDO; Sidanius and Pratto, 2001). SDO is assessed on the same individuals from
1996 to 2000 (Sidanius et al., 2010), involving the following four items, scored on a 7-point scale:  

* It is probably a good thing that certain groups are at the top and other groups are at the bottom (I1).

* Inferior groups should stay in their place (I2).

* We should do what we can to equalize conditions for different groups (I3, reversed).

* Increased social equality is beneficial to society (I4, reversed).

For the moment, let us consider a simple latent variable structure where all four
items load on a general SDO dimension. We pick 2 years only: 1996 vs. 1998.  

The most relevant model within this context is the **strong/scale invariance model**, which allows us to compare the latent means across the two time points. We restrict the factor loadings as well as the corresponding intercepts of each item to be equal in both measurement occasions (**strong/scale invariance**). Also, we need to allow for residual covariances (using the `~~` symbol in the syntax) for each item across time points. With this we account for autoregressive effects, which has to be done when modeling longitudinal data.

### Fit the models

We will start with the scale invariance model and work our way down to lower levels of measurement invariance. (We exclude the strict invariance model here).

We will start with the scale invariance model and work our way down to lower levels of measurement invariance. (We exclude the strict invariance model here).

#### Scale Invariance

In [20]:
# Specify the model
ro.r('model_scale <- "'
      'SDO1996 =~ 1*I1.1996 + a2*I2.1996 + a3*I3.1996 + a4*I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + a2*I2.1998 + a3*I3.1998 + a4*I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ int1*1; I1.1998 ~ int1*1\n'
      'I2.1996 ~ int2*1; I2.1998 ~ int2*1\n'
      'I3.1996 ~ int3*1; I3.1998 ~ int3*1\n'
      'I4.1996 ~ int4*1; I4.1998 ~ int4*1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 1"')

# Fit the model
ro.r('fit_scale <- lavaan::cfa(model_scale, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_scale = ro.r('summary(fit_scale, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_scale)

lavaan 0.6-19 ended normally after 45 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        30
  Number of equality constraints                     7

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               315.748     203.519
  Degrees of freedom                                21          21
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.551
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correcti

The parameters of main interest are the latent variable means (however, this has nothing to do with measurement invariance but can be used to compare latent score bewteen time points):

In [26]:
print(ro.r("parameterEstimates(fit_scale)[22:23,]"))

       lhs op rhs label    est    se      z pvalue ci.lower ci.upper
22 SDO1996 ~1            0.000 0.000     NA     NA    0.000    0.000
23 SDO1998 ~1           -0.047 0.019 -2.432  0.015   -0.085   -0.009



We see that the mean for 1996 was fixed to 0 and is therefore used as the reference
year. The second line suggests that there is a significant decrease in SDO from 1996
to 1998.  

Note that a **weak/metric invariance** version of this model can be fitted by *restricting both*
*latent means to 0* and *freeing up the intercepts*. For a **configural invariance** version,
the *loadings need to be freed up as well* (first two syntax lines). For a **strict invariance** version, *intercepts, loadings and residuals* need to be *fixed*.  
If we do not want to do this by hand, the `longInvariance` function from `semTools` can be used
to establish such a generic sequence (similar to the `measurementInvariance` function used above).
Using the `anova` function, the models can be again tested against each other.  

Let us fit a **weak/metric invariance** model.

#### Metric Invariance

Adapt the code from the scale model to fit a metric model by freeing up the intercepts and restrict both latent variables to zero. Name your model `model.metric`.

In [27]:
# Specify the model
ro.r('model_metric <- "'
      'SDO1996 =~ 1*I1.1996 + a2*I2.1996 + a3*I3.1996 + a4*I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + a2*I2.1998 + a3*I3.1998 + a4*I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ 1; I1.1998 ~ 1\n'
      'I2.1996 ~ 1; I2.1998 ~ 1\n'
      'I3.1996 ~ 1; I3.1998 ~ 1\n'
      'I4.1996 ~ 1; I4.1998 ~ 1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 0*1"')

# Fit the model
ro.r('fit_metric <- lavaan::cfa(model_metric, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_metric = ro.r('summary(fit_metric, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_metric)

lavaan 0.6-19 ended normally after 43 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        29
  Number of equality constraints                     3

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               310.762     189.360
  Degrees of freedom                                18          18
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.641
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correcti

Note that the values in the `intercepts` section are now varying.

Again, we can extract latent variable means (however, this has again nothing to do with measurement invariance):

In [28]:
print(ro.r("parameterEstimates(fit_metric)[22:23,]"))

       lhs op rhs label est se  z pvalue ci.lower ci.upper
22 SDO1996 ~1             0  0 NA     NA        0        0
23 SDO1998 ~1             0  0 NA     NA        0        0



Let us continue with a **configural variance** model.

#### Configural Invariance

Adapt the code from the metric model to fit a configural model by freeing up the loadings as well (first two syntax lines). Name your model `model.configural`.

In [29]:
# Specify the model
ro.r('model_configural <- "'
      'SDO1996 =~ 1*I1.1996 + I2.1996 + I3.1996 + I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + I2.1998 + I3.1998 + I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ 1; I1.1998 ~ 1\n'
      'I2.1996 ~ 1; I2.1998 ~ 1\n'
      'I3.1996 ~ 1; I3.1998 ~ 1\n'
      'I4.1996 ~ 1; I4.1998 ~ 1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 0*1"')

# Fit the model
ro.r('fit_configural <- lavaan::cfa(model_configural, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_configural = ro.r('summary(fit_configural, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_configural)

lavaan 0.6-19 ended normally after 61 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        29

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               301.687     175.637
  Degrees of freedom                                15          15
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.718
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correction factor                                  1.816

User 

#### Model Comparison

To assess which model provides a better fit (and therefore to assess which level of measurement invariance holds) we can use the `anova()` function.  

Let us first compare the **configural model** and the **metric model**.

In [30]:
print(ro.r("anova(fit_configural, fit_metric)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
               Df   AIC   BIC  Chisq Chisq diff Df diff Pr(>Chisq)  
fit_configural 15 14812 14940 301.69                                
fit_metric     18 14815 14930 310.76     7.2119       3    0.06544 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



We see that the metric model fits worse than the configural model. This is no suprise as we include more restrictions in the metric model. However, the fit is not significantly worse (p = .0654) and the BIC even favors the metric model. From this we may conclude that metric invariance holds but there is a huge BUT (see below).

Next, lets compare the **metric model** and the **scale model**.

In [32]:
print(ro.r("anova(fit_metric, fit_scale)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
           Df   AIC   BIC  Chisq Chisq diff Df diff Pr(>Chisq)
fit_metric 18 14815 14930 310.76                              
fit_scale  21 14814 14916 315.75     4.9207       3     0.1777



From this comparison we can conclude that scale invariance holds as the scale model is not significantly worse than the metric model.  

Now to the **BUT**: You can see in the model output of the **configural model** that the fit is quite bad (e.g. RMSEA =  0.177 (> .08)). This is not too surprising since research has shown that there are two subdimensions of SDO (Ho et al., 2015):
anti-egalitarianism and dominance. According to the theory, the first two items are
supposed to load on the dominance dimension (SDO-D), whereas the remaining two
items load on anti-egalitarianism (SDO-E).  

Therefore, we should be cautious with interpreting the comparisons above and first make sure that we find a good fitting model to our data.